In [1]:
import numpy as np
import logging 
import sys
import os
import h5py
import math
from math import radians, cos, sin
from scipy.cluster.vq import whiten, kmeans, kmeans2

logger = logging.getLogger(__name__)
disp=False

In [2]:
### read the data we have collected
with h5py.File('../scripts/{}.h5'.format('joints_data'), 'r+') as f:
    pos_grp = f['workspace_coords']
    q       = pos_grp['joint_positions'].value
    qdot    = pos_grp['joint_velocities'].value
    targ_grp = f['workspace_targets']
    qstar   = targ_grp['joint_positions'].value
    qdot_star= targ_grp['joint_velocities'].value

In [3]:
q1 = q[0:7,:]
q2 = q[7:14,:]
q1dot = qdot[0:7,:]
q2dot = qdot[7:14,:]
pts = dict(pts1=dict(q=q1, qdot=q1dot),pts2=dict(q=q2, qdot=q2dot),targets=dict(q=qstar, qdot=qdot_star),)

In [4]:
np.set_printoptions(suppress=True)

if disp:
    print(pts)

### convert from joint space to cartesian coordinates in the end effector/tool frame

In [5]:
#### defining D-H constants
"""lc, lf are in millimeters"""
lc, lf = 300, 350

pts_cartesian = {
    'pts1': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    },
    'pts2': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    },
    'targets': {
        'x': None, 'y': None, 'z': None,
        'xdot': None, 'ydot': None, 'zdot': None,
    }
}

In [6]:
pts['pts1'].keys()

dict_keys(['qdot', 'q'])

In [7]:
way_pts = [k for k, v in pts.items()]
for way_pt in way_pts:
    # get the positional points
    px_l = [cos(x) for x in pts[way_pt]['q'][:,0]]
    px_r1 = [lc*cos(x) for x in  pts[way_pt]['q'][:,1]]
    px_r2 = [lf*cos(x+y) for x,y in  zip(pts[way_pt]['q'][:,1], pts[way_pt]['q'][:,2])]
    px_r = np.array(px_r1) - px_r2
    pts_cartesian[way_pt]['x'] = np.array(px_l) * np.array(px_r)

    py_l = [sin(x) for x in pts[way_pt]['q'][:,0]]
    pts_cartesian[way_pt]['y'] = np.array(py_l) * np.array(px_r)

    pz_l = [-lc * sin(x) for x in pts[way_pt]['q'][:,1]]
    pz_r = [lf * cos(x+y) for x, y in zip(pts[way_pt]['q'][:,1], pts[way_pt]['q'][:,2])]
    pts_cartesian[way_pt]['z'] = np.array(pz_l) - np.array(pz_r)
    
    # get the velocities
    px_l = [cos(x) for x in pts[way_pt]['qdot'][:,0]]
    px_r1 = [lc*cos(x) for x in  pts[way_pt]['qdot'][:,1]]
    px_r2 = [lf*cos(x+y) for x,y in  zip(pts[way_pt]['qdot'][:,1], pts[way_pt]['qdot'][:,2])]
    px_r = np.array(px_r1) - px_r2
    pts_cartesian[way_pt]['xdot'] = np.array(px_l) * np.array(px_r)

    py_l = [sin(x) for x in pts[way_pt]['qdot'][:,0]]
    pts_cartesian[way_pt]['ydot'] = np.array(py_l) * np.array(px_r)

    pz_l = [-lc * sin(x) for x in pts[way_pt]['qdot'][:,1]]
    pz_r = [lf * cos(x+y) for x, y in zip(pts[way_pt]['qdot'][:,1], pts[way_pt]['qdot'][:,2])]
    pts_cartesian[way_pt]['zdot'] = np.array(pz_l) - np.array(pz_r)
    
# print(pts_cartesian)

#### gather all the cartesian points and velocities
+ data is thus arranged:
  - data = [
             \zeta^0, \zeta^1, \zeta^\star; 
             \dot{\zeta^0}, \dot{\zeta^1}, \dot{\zeta^\star}
            ]
  - where \zeta \in R^n, n being the dimension in cartesian coordinates of the 
  - note that our data is in 2D for now, i.e. x and y axes in Cartesian space
  
  In other words, data is thus shaped:
                  
  data = 
                  time t1                                   time t2                          time t3                               time t4
         [x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 | x1,  x2,  x3,  x4,  x5,  x6,  x7 ]
         [y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 | y1,  y2,  y3,  y4,  y5,  y6,  y7 ]
         [x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d| x1d, x2d, x3d, x4d, x5d, x6d, x7d]
         [y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d| y1d, y2d, y3d, y4d, y5d, y6d, y7d]

In [8]:
x = np.hstack([pts_cartesian['pts1']['x'], pts_cartesian['pts2']['x'], pts_cartesian['targets']['x']])
y = np.hstack([pts_cartesian['pts1']['y'], pts_cartesian['pts2']['y'], pts_cartesian['targets']['y']])
z = np.hstack([pts_cartesian['pts1']['z'], pts_cartesian['pts2']['z'], pts_cartesian['targets']['z']])

# xd = np.hstack([pts_cartesian['pts1']['xdot'], pts_cartesian['pts2']['xdot'], pts_cartesian['targets']['xdot']])
# yd = np.hstack([pts_cartesian['pts1']['ydot'], pts_cartesian['pts2']['ydot'], pts_cartesian['targets']['ydot']])
# zd = np.hstack([pts_cartesian['pts1']['zdot'], pts_cartesian['pts2']['zdot'], pts_cartesian['targets']['zdot']])
xd,yd,zd = [[] for _ in range(3)]
for i in range(len(x)-1):
    xd.append((x[i+1]-x[i])/0.1)
    yd.append((y[i+1]-y[i])/0.1)
    zd.append((z[i+1]-z[i])/0.1)
xd.append(0)
yd.append(0)
zd.append(0)

In [9]:
# x.shape, y.shape, z.shape
data = np.vstack([x,y,np.array(xd),np.array(yd)])
#data = np.vstack([x,y,z])
# print(data.shape)
filename = '../scripts/{}.h5'.format('torobo_processed_data')
os.remove(filename) if os.path.isfile(filename) else None
# time.sleep(4)
with h5py.File(filename, 'w') as f:
    pos_grp = f.create_group('data')
    pos_grp.create_dataset("data", data=data, dtype=np.float32, compression="gzip", compression_opts=9)

d = data.shape[0]/2
num_states = 4
num_var, num_data = data.shape
centers, labels = kmeans_init_data(data, num_states=num_states)
mu = centers.T

print('labels: ', labels)
print('centers: ', centers)
Priors = np.zeros((num_states))
Sigma  = np.zeros((num_states, num_states, num_states))

for i in range(num_states):
    idtemp = np.nonzero(labels==i)
    Priors[i] = len(idtemp[0])
    print('Priors: ', Priors[i])
    print('idtemp[0]: ', idtemp[0], 'labels[idtemp]: ', labels[idtemp])
    if labels[idtemp].size == 0:
        Sigma[:,:,i] = np.random.randn(num_states, num_states)
    else:
        Sigma[:,:,i] = np.cov(m=data[:, idtemp[0]])
    print('data[:, idtemp[0]]: ', data[:, idtemp[0]].T)
    print('Sigma[:,:,i]: \n', Sigma[:,:,i])
    print(1e-5 * np.diag(np.ones((num_var, 1))))
    # avoid numerical stability
    Sigma[:,:,i] = Sigma[:,:,i] + 1e-5 * np.diag(np.ones([num_var, 1]))

Priors = np.divide(Priors, sum(Priors))